In [ ]:
import warnings

import os.path as op
import numpy as np
import cv2

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from ordering_methods import *
from statistical_analysis import *

In [ ]:
# prefix = "/work/scratch/malinoro/histogram_transfer/data"
prefix = "/Users/roman/Code/Histogram_transfer/data/"

# Reading or creating images

In [ ]:
img1 = cv2.cvtColor(cv2.imread(op.join(prefix, "MiddleBury", "cones.ppm")), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(op.join(prefix, "MiddleBury", "teddy.ppm")), cv2.COLOR_BGR2RGB)
img1 = cv2.resize(img1, dsize=(360, 300))
img2 = cv2.resize(img2, dsize=(360, 300))

## Fake image
X, Y = np.meshgrid(np.linspace(0, 255, img1.shape[1]), np.linspace(0, 255, img1.shape[0]))
R, G, B = np.sqrt((X**2+Y**2)/2), np.sqrt(((255-X)**2+Y**2)/2), np.sqrt(((255-X)**2+(255-Y)**2)/2)
warnings.warn("Using numpy.astype method which always work but can be dependent of OS and other obscure stuff.\nBe sure to check that it casts correctly on your computer")
R, G, B = R.astype(dtype=np.uint8), G.astype(dtype=np.uint8), B.astype(dtype=np.uint8)
img0 = cv2.merge([R, G, B])

# Choosing and image, smoothing it, HSV, and separating channels

In [ ]:
img = img1
img_ref = img2

# Smooth image
img_smooth = cv2.GaussianBlur(img, (5, 5), 0)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 5))

axes[0,0].imshow(img_ref)
axes[0,0].set_title("Reference image")
axes[0,1].imshow(img)
axes[0,1].set_title("Image to transfer")
axes[0,2].imshow(img_smooth)
axes[0,2].set_title("Smoothed image")

n_bins = 80
for k, c in zip(range(3), ["r", "g", "b"]):
    counts, bins = np.histogram(img_ref[:,:,k], bins=n_bins, density=True)
    axes[1,0].stairs(counts, bins, color=c)
    counts, bins = np.histogram(img[:,:,k], bins=n_bins, density=True)
    axes[1,1].stairs(counts, bins, color=c)
    counts, bins = np.histogram(img_smooth[:,:,k], bins=n_bins, density=True)
    axes[1,2].stairs(counts, bins, color=c)
    
plt.show()

In [ ]:
cumulated_hist = False

bins_3d = (20, 20, 20)
X, Y, Z = np.meshgrid(*[np.linspace(0, 255, b) for b in bins_3d])

# This compute the histogram in N dimensions
if cumulated_hist:
    counts, hist = np.histogramdd(np.array(img).reshape(-1, 3), bins=bins_3d)
    counts = counts.cumsum(axis=0).cumsum(axis=1).cumsum(axis=2) / np.sum(counts)
    iso_min, iso_max = 0, 1
else:
    counts, hist = np.histogramdd(np.array(img).reshape(-1, 3), bins=bins_3d, density=True)
    iso_min, iso_max = np.quantile(counts, 0.5), counts.max()

    
fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    isomin=iso_min,
    isomax=iso_max,
    value=counts.flatten(),
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))

fig.update_layout(autosize=False, width=500, height=500, scene=dict(xaxis_title='Red', yaxis_title='Green', zaxis_title='Blue'))
fig.show()

# Different color spaces

In [ ]:
image_RGB = img
image_LAB = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2LAB)
image_HSV = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2HSV)
image_YCrCb = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2YCrCb)
image_LUV = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2Luv)
image_XYZ = cv2.cvtColor(image_RGB, cv2.COLOR_RGB2XYZ)
image_ruderman = cvt_ruderman(image_XYZ)


image_ref_RGB = img_ref
image_ref_LAB = cv2.cvtColor(image_ref_RGB, cv2.COLOR_RGB2LAB)
image_ref_HSV = cv2.cvtColor(image_ref_RGB, cv2.COLOR_RGB2HSV)
image_ref_YCrCb = cv2.cvtColor(image_ref_RGB, cv2.COLOR_RGB2YCrCb)
image_ref_LUV = cv2.cvtColor(image_ref_RGB, cv2.COLOR_RGB2Luv)
image_ref_XYZ = cv2.cvtColor(image_ref_RGB, cv2.COLOR_RGB2XYZ)
image_ref_ruderman = cvt_ruderman(image_ref_XYZ)


out_RGB = transfer_independent_channels(image_RGB, image_ref_RGB)
out_LAB = transfer_independent_channels(image_LAB, image_ref_LAB)
out_HSV = transfer_independent_channels(image_HSV, image_ref_HSV)
out_YCrCb = transfer_independent_channels(image_YCrCb, image_ref_YCrCb)
out_LUV = transfer_independent_channels(image_LUV, image_ref_LUV)
out_XYZ = transfer_independent_channels(image_XYZ, image_ref_XYZ)
out_ruderman = transfer_independent_channels(image_ruderman, image_ref_ruderman)
out_ruderman = cvt_back_ruderman(out_ruderman)
out_ruderman[np.nonzero(~(out_ruderman>0))] = 0  # removes nan
out_ruderman = np.round(out_ruderman).astype(dtype=out_XYZ.dtype)


out_LAB_RGB = cv2.cvtColor(out_LAB, cv2.COLOR_LAB2RGB)
out_HSV_RGB = cv2.cvtColor(out_HSV, cv2.COLOR_HSV2RGB)
out_YCrCb_RGB = cv2.cvtColor(out_YCrCb, cv2.COLOR_YCrCb2RGB)
out_LUV_RGB = cv2.cvtColor(out_LUV, cv2.COLOR_LUV2RGB)
out_XYZ_RGB = cv2.cvtColor(out_XYZ, cv2.COLOR_XYZ2RGB)
out_ruderman_RGB = cv2.cvtColor(out_ruderman, cv2.COLOR_XYZ2RGB)

In [ ]:
fig, axis = plt.subplots(nrows=3, ncols=3, figsize=(10, 10))
axis[0,0].imshow(img_ref)
axis[0,0].set_title("Image Ref")
axis[0,1].imshow(img)
axis[0,1].set_title("Image")
axis[0,2].imshow(out_RGB)
axis[0,2].set_title("Transfer on BGR")
axis[1,0].imshow(out_LAB_RGB)
axis[1,0].set_title("Transfer on LAB")
axis[1,1].imshow(out_HSV_RGB)
axis[1,1].set_title("Transfer on HSV")
axis[1,2].imshow(out_YCrCb_RGB)
axis[1,2].set_title("Transfer on YCrCb")
axis[2,0].imshow(out_LUV_RGB)
axis[2,0].set_title("Transfer on LUV")
axis[2,1].imshow(out_XYZ_RGB)
axis[2,1].set_title("Transfer on XYZ")
axis[2,2].imshow(out_ruderman_RGB)
axis[2,2].set_title("Transfer on Ruderman");

# TEMP

In [ ]:
import pandas as pd

In [ ]:
img = cv2.cvtColor(img1, cv2.COLOR_RGB2XYZ)
img_ref = cv2.cvtColor(img2, cv2.COLOR_RGB2XYZ)

In [ ]:
kernel_size=5

img_smooth = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
img_ref_smooth = cv2.GaussianBlur(img_ref, (kernel_size, kernel_size), 0)

if img.dtype == np.dtype('uint8'):
    warnings.warn("img dtype is unsigned integer. Casting to int32 to compute euclidian distance")
    img = img.astype(dtype=np.int32)
    img_smooth = img_smooth.astype(dtype=np.int32)

if img_ref.dtype == np.dtype('uint8'):
    warnings.warn("img_ref dtype is unsigned integer. Casting to int32 to compute euclidian distance")
    img_ref = img_ref.astype(dtype=np.int32)
    img_ref_smooth = img_ref_smooth.astype(dtype=np.int32)

In [ ]:
edges_pivot_points = [[0, 0, 0], [255, 0, 0], [0, 255, 0], [0, 0, 255],
                      [255, 255, 0], [255, 0, 255], [0, 255, 255], [255, 255, 255],
                      [127, 0, 0], [127, 255, 0], [127, 0, 255], [127, 255, 255],
                      [0, 127, 0], [255, 127, 0], [0, 127, 255], [255, 127, 255],
                      [0, 0, 127], [255, 0, 127], [0, 255, 127], [255, 255, 127]]

df_edges = pd.DataFrame(columns=["E"+str(i) for i in range(len(edges_pivot_points))])
df_ref_edges = pd.DataFrame(columns=["E"+str(i) for i in range(len(edges_pivot_points))])

In [ ]:
for i, p in enumerate(edges_pivot_points):
        df = pd.DataFrame()
        df_ref = pd.DataFrame()
        df["D_img"] = np.square(img.reshape((-1, 3)) - np.array(p)).sum(axis=1)
        df["D_smooth"] = np.square(img_smooth.reshape((-1, 3)) - np.array(p)).sum(axis=1)
        df_ref["D_img"] = np.square(img_ref.reshape((-1, 3)) - np.array(p)).sum(axis=1)
        df_ref["D_smooth"] = np.square(img_ref_smooth.reshape((-1, 3)) - np.array(p)).sum(axis=1)

        df = df.sort_values(by=["D_img", "D_smooth"])
        df_ref = df_ref.sort_values(by=["D_img", "D_smooth"])

        df_edges["E" + str(i)] = df.index
        df_ref_edges["E" + str(i)] = df_ref.index

In [ ]:
order_with_duplicates = ~pd.Series(df_edges.to_numpy().flatten()).duplicated().to_numpy().reshape(df_edges.shape)
order_ref_with_duplicates = ~pd.Series(df_ref_edges.to_numpy().flatten()).duplicated().to_numpy().reshape(df_ref_edges.shape)

In [ ]:
n_pixels = img.shape[0]*img.shape[1]
for i in range(len(edges_pivot_points)):
    nan_tail = np.array((n_pixels - order_with_duplicates[:, i].sum())*[n_pixels])
    df_edges["E" + str(i)] = np.append(df_edges["E" + str(i)][order_with_duplicates[:, i]].to_numpy(), nan_tail)

    nan_tail = np.array((n_pixels - order_ref_with_duplicates[:, i].sum())*[n_pixels])
    df_ref_edges["E" + str(i)] = np.append(df_ref_edges["E" + str(i)][order_ref_with_duplicates[:, i]].to_numpy(), nan_tail)

In [ ]:
order = df_edges.to_numpy().ravel()
order = np.unravel_index(order[order != n_pixels], img.shape[:2])

order_ref = df_ref_edges.to_numpy().ravel()
order_ref = np.unravel_index(order_ref[order_ref != n_pixels], img_ref.shape[:2])

img_copy = img.copy()
img_copy[order] = img_ref[order_ref]

In [ ]:
img_copy = cv2.cvtColor(img_copy.astype(dtype=np.uint8), cv2.COLOR_XYZ2RGB)
img_ref = img2

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

axes[0, 0].imshow(img_ref)
axes[0, 1].imshow(img_copy)

n_bins = 80

for k, c in enumerate(["r", "g", "b"]):
    counts, bins = np.histogram(img_ref[:,:,k], bins=n_bins, density=True)
    axes[1,0].stairs(counts, bins, color=c)
    counts, bins = np.histogram(img_copy[:,:,k], bins=n_bins, density=True)
    axes[1,1].stairs(counts, bins, color=c)

bins_3d = (20, 20, 20)
counts_ref, hist_ref = np.histogramdd(np.array(img_ref).reshape(-1, 3), bins=bins_3d, density=True)
counts, hist = np.histogramdd(np.array(img_copy).reshape(-1, 3), bins=bins_3d, density=True)
(counts_ref == counts).all() & (np.array(hist)==np.array(hist_ref)).all()